In [2]:
%matplotlib inline
import os
import glob
import numpy as np
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Input, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.visualize_util import plot
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from matplotlib import pyplot as plt


Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [6]:
def data_list():
    folder = './urban_tribe/pictures_all/'
    imgs = glob.glob1(folder, '*.jpg')
    dict = {}
    for img in imgs:
        c = img.split('_')[0]
        if c not in dict:
            dict[c] = list()
        img = os.path.join(folder, img)
        dict.get(c).append(img)
    train = []
    val = []
    test = []
    for c in dict.iterkeys():
        if c == 'others': continue
        l = np.array(dict[c])
        idx = range(len(dict[c]))
        np.random.shuffle(idx)
        train.append(l[idx[: len(idx) * 8 / 10]].tolist())
        val.append(l[idx[len(idx) * 8 / 10: len(idx) * 9 / 10]].tolist())
        test.append(l[idx[len(idx) * 9 / 10:]].tolist())
    return train, val, test

#read from list
def img_read(train_data, num):
    train_img = []
    train_label = []
    for i in range(len(train_data)):
        label = np.zeros(10)
        label[i] = 1
        imagenum = 0
        while imagenum < num and imagenum < len(train_data[i]):
            img = load_img(train_data[i][imagenum], target_size=(224, 224))
            x = img_to_array(img, dim_ordering='tf')
            x = x.reshape((1,) + x.shape)
            train_label.append(label)
            train_img.append(x)
            imagenum += 1
    train_img = np.vstack(train_img)
    train_label = np.vstack(train_label)
    return train_img, train_label


In [4]:
def getModel( output_dim ):
    vgg_model = VGG16( weights='imagenet', include_top=True )
    
    vgg_out = vgg_model.layers[-2].output #Last FC layer's output 
    vgg_out.trainable = False
    softmax_layer = Dense(output_dim=output_dim, activation='softmax')(vgg_out)
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    for layer in vgg_model.layers:
        layer.trainable = False
    #Confirm the model is appropriate
    tl_model.compile(optimizer='RMSprop', loss='categorical_crossentropy')
    #tl_model.summary()
    return tl_model

In [ ]:
def main():
    #Output dim for your dataset
    output_dim = 10 #For Prom2
    tl_model = getModel( output_dim ) 
    #plot(tl_model, to_file='model.png')
    train_data, val_data, test_data = data_list()
    train_img, train_label = img_read(train_data, 200)
    train_img = preprocess_input(train_img)
    
    val_img, val_label = img_read(val_data, 10)
    val_img = preprocess_input(val_img)
    # Train the model
    # fit(x, y, batch_size=32, nb_epoch=10, verbose=1, callbacks=None, 
    #    validation_split=0.0, validation_data=None, shuffle=True, 
    #    class_weight=None, sample_weight=None, initial_epoch=0)
    tl_model.fit(train_img, train_label, batch_size=10, nb_epoch=10, validation_data=(val_img, val_label))
    # tl_model.fit_generator(train, 2000, 50)
    # Test the model

if __name__ == '__main__':
    main()


Train on 823 samples, validate on 99 samples
Epoch 1/10
823/823 [==============================] - 792s - loss: 2.0240 - val_loss: 1.3395